In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [8]:
dates = ['1594512000','1594598400','1594684800']
id = '239'
sessUrl = 'https://professionalabstracts.com/api/iplanner/index.php?conf=isth2020&method=get&model=sessions&params%5Bsids%5D='+id+'&secret=true'
daySessionsUrl = 'https://professionalabstracts.com/api/iplanner/index.php?conf=isth2020&method=get&model=day&params%5Bday%5D='+dates[1]+'&params%5Border%5D=&params%5Bsearch%5D=&secret=true'
onDemandPosterSessions = 'https://professionalabstracts.com/api/iplanner/index.php?conf=isth2020&method=get&model=sessions&params%5Bsids%5D=320,321,322,323,324,325,326,327,328,329,330,331,332,333&secret=true'
presId = '2309'
presentationData = 'https://professionalabstracts.com/api/iplanner/index.php?conf=isth2020&method=get&model=presentation&params%5Bpid%5D='+presId+'&secret=true'

In [16]:
data = requests.get(daySessionsUrl)

In [17]:
dayData = data.json()

In [21]:
dayData['sessions'][dates[1]]

[{'id': '289',
  'color': '#FFFFFF',
  'color_rgb': [255, 255, 255],
  'background': '#005cab',
  'background_rgb': [0, 92, 171],
  'start': 480,
  'length': 30,
  'time': '08:00-08:30 EDT',
  'code': '',
  'title': 'Plenary Session',
  'room': 34,
  'overlaps': False},
 {'id': '297',
  'color': '#FFFFFF',
  'color_rgb': [255, 255, 255],
  'background': '#003366',
  'background_rgb': [0, 51, 102],
  'start': 525,
  'length': 75,
  'time': '08:45-10:00 EDT',
  'code': '',
  'title': 'State-of-the-Art on Diagnostics and OMICs',
  'room': 34,
  'overlaps': False},
 {'id': '298',
  'color': '#FFFFFF',
  'color_rgb': [255, 255, 255],
  'background': '#003366',
  'background_rgb': [0, 51, 102],
  'start': 525,
  'length': 75,
  'time': '08:45-10:00 EDT',
  'code': '',
  'title': 'State-of-the-Art on Platelet Disorders and von Willebrand Disease',
  'room': 35,
  'overlaps': False},
 {'id': '299',
  'color': '#FFFFFF',
  'color_rgb': [255, 255, 255],
  'background': '#003366',
  'background_r

In [7]:
ids = []
for i in range(len(dates)):
    daySessionsUrl = 'https://professionalabstracts.com/api/iplanner/index.php?conf=isth2020&method=get&model=day&params%5Bday%5D='+dates[i]+'&params%5Border%5D=&params%5Bsearch%5D=&secret=true'
    data = requests.get(daySessionsUrl)
    dayData = data.json()
    sessionList = dayData['sessions'][dates[i]]
    print(dates[i] , " " ,len(sessionList))
    for j in sessionList:
        ids.append(j['id'])
        

1594512000   18
1594598400   16
1594684800   28


In [11]:
dataPosters = requests.get(onDemandPosterSessions)
posters = dataPosters.json()

In [17]:
for i in posters:
    ids.append(posters[i]['id'])

In [6]:
sessUrl = 'https://professionalabstracts.com/api/iplanner/index.php?conf=isth2020&method=get&model=sessions&params%5Bsids%5D='+'320'+'&secret=true'
data = requests.get(sessUrl)
session = data.json()
session['1']

{'id': 320,
 'layout': 'code-title',
 'color': '#000000',
 'background': '#019961',
 'group': '',
 'type': 'On-Demand Poster Session',
 'date': '12.07.2020',
 'time': '14:00 – 15:00 EDT',
 'code': '',
 'title': 'Arterial Thromboembolism Abstracts',
 'titleplain': 'Arterial Thromboembolism Abstracts',
 'outline': 'The full abstract text will be available on June 29, 2020 at 09:00 AM U.S. Eastern Time.<br />\r\nThe ePoster will be available on the ISTH 2020 Virtual Congress Platform starting July 12, 2020.<br />',
 'texttop': '',
 'textmiddle': '',
 'textbottom': '',
 'room': 'Virtual Meeting Room 6',
 'pres': {'1': {'id': 2106,
   'time': '',
   'frame': '14:00-15:00',
   'code': '',
   'type': 2,
   'abs': '',
   'title': 'Acute Coronary Syndromes',
   'titleplain': 'Acute Coronary Syndromes',
   'outline': '',
   'highlight': False,
   'aut': '',
   'inst': '',
   'search': None},
  '2': {'id': 2309,
   'time': '',
   'frame': '14:00-15:00',
   'code': 'PB0001',
   'type': 0,
   'abs'

In [25]:
dfPresDetails = pd.DataFrame()
for id_ in tqdm(ids):
    
    sessUrl = 'https://professionalabstracts.com/api/iplanner/index.php?conf=isth2020&method=get&model=sessions&params%5Bsids%5D='+str(id_)+'&secret=true'
    data = requests.get(sessUrl)
    session = data.json()
    session = session['1']
    try:
        pres = session['pres']
    except:
        pres = {}
    
    if len(pres) == 0:
        details = {}
        details['Session Time'] = session['time']
        details['Session Title'] = session['title']
        details['Session Date'] = session['date']
        details['Session Location'] = session['room']
        dfPresDetails = dfPresDetails.append(details,ignore_index = True)
    else:
        for i in pres:
            details = {}
            details['Session Time'] = session['time']
            details['Session Title'] = session['title']
            details['Session Date'] = session['date']
            details['Session Location'] = session['room']
            details['Presentation Title'] = pres[i]['title']
            details['Presentation Start Time'] = pres[i]['time']
            details['Presentation Time'] = pres[i]['frame']
            try:
                details['Abstract Code'] = pres[i]['code']
            except:
                pass
            try:
                details['Authors'] = [pres[i]['pers'][j]['text'] for j in pres[i]['pers']]
            except:
                try:
                    details['Authors'] = pres[i]['aut']
                except:
                    pass
            try:
                details['Institution'] =  pres[i]['inst']
            except:
                pass
            print(details)
            dfPresDetails = dfPresDetails.append(details,ignore_index = True)
            
            
            
            
            
        
        
    


{'Session Time': '08:00 – 08:30 EDT', 'Session Title': 'Plenary Session', 'Session Date': '12.07.2020', 'Session Location': 'Virtual Meeting Room 1', 'Presentation Title': 'Advances in Gene Therapy and Ethical Aspects', 'Presentation Start Time': '08:00', 'Presentation Time': '08:00-08:30', 'Abstract Code': '', 'Authors': ['Speaker: Luigi Naldini, Italy'], 'Institution': ''}
{'Session Time': '08:45 – 10:00 EDT', 'Session Title': 'State-of-the-Art on Hemophilia and Rare Bleeding Disorders', 'Session Date': '12.07.2020', 'Session Location': 'Virtual Meeting Room 1', 'Presentation Title': 'Progress and Obstacles to Hemophilia Gene Therapy', 'Presentation Start Time': '08:45', 'Presentation Time': '08:45-09:10', 'Abstract Code': '', 'Authors': ['Speaker: Amit Nathwani, United Kingdom'], 'Institution': ''}
{'Session Time': '08:45 – 10:00 EDT', 'Session Title': 'State-of-the-Art on Hemophilia and Rare Bleeding Disorders', 'Session Date': '12.07.2020', 'Session Location': 'Virtual Meeting Roo

{'Session Time': '13:15 – 13:45 EDT', 'Session Title': 'Plenary Session', 'Session Date': '12.07.2020', 'Session Location': 'Virtual Meeting Room 1', 'Presentation Title': 'VWF in Hemostasis: From Basic Science to Clinic', 'Presentation Start Time': '13:15', 'Presentation Time': '13:15-13:45', 'Abstract Code': '', 'Authors': ['Speaker: Peter J. Lenting, France'], 'Institution': ''}
{'Session Time': '08:00 – 08:30 EDT', 'Session Title': 'Plenary Session', 'Session Date': '13.07.2020', 'Session Location': 'Virtual Meeting Room 1', 'Presentation Title': 'Unmet Clinical Needs and Future Advances in VTE', 'Presentation Start Time': '08:00', 'Presentation Time': '08:00-08:30', 'Abstract Code': '', 'Authors': ['Speaker: Alexander Cohen, United Kingdom'], 'Institution': ''}
{'Session Time': '08:45 – 10:00 EDT', 'Session Title': 'State-of-the-Art on Diagnostics and OMICs', 'Session Date': '13.07.2020', 'Session Location': 'Virtual Meeting Room 1', 'Presentation Title': 'Genome-Wide Polygenic Sc

{'Session Time': '08:45 – 10:00 EDT', 'Session Title': 'State-of-the-Art on Venous Thromboembolism', 'Session Date': '13.07.2020', 'Session Location': 'Virtual Meeting Room 7', 'Presentation Title': 'Managing DOACs in Emergencies', 'Presentation Start Time': '08:45', 'Presentation Time': '08:45-09:10', 'Abstract Code': '', 'Authors': ['Speaker: Cecilia Becattini, Italy'], 'Institution': ''}
{'Session Time': '08:45 – 10:00 EDT', 'Session Title': 'State-of-the-Art on Venous Thromboembolism', 'Session Date': '13.07.2020', 'Session Location': 'Virtual Meeting Room 7', 'Presentation Title': 'New Antithrombotics and New Interventions for VTE Beyond Anti-IIa and Anti-Xa', 'Presentation Start Time': '09:10', 'Presentation Time': '09:10-09:35', 'Abstract Code': '', 'Authors': ['Speaker: Stavros Konstantinides, Germany'], 'Institution': ''}
{'Session Time': '08:45 – 10:00 EDT', 'Session Title': 'State-of-the-Art on Venous Thromboembolism', 'Session Date': '13.07.2020', 'Session Location': 'Virtu

{'Session Time': '14:00 – 15:00 EDT', 'Session Title': 'Arterial Thromboembolism Abstracts', 'Session Date': '12.07.2020', 'Session Location': 'Virtual Meeting Room 6', 'Presentation Title': 'Atherosclerosis', 'Presentation Start Time': '', 'Presentation Time': '14:00-15:00', 'Abstract Code': '', 'Authors': '', 'Institution': ''}
{'Session Time': '14:00 – 15:00 EDT', 'Session Title': 'Arterial Thromboembolism Abstracts', 'Session Date': '12.07.2020', 'Session Location': 'Virtual Meeting Room 6', 'Presentation Title': 'High-Dose Atorvastatin – Loaded Microbubbles-Mediated Sonoporation Therapy using B - Mode Ultrasound - Guided Focused Electrohydraulic Shock Waves Decreases Inflammation and Restenosis after Laser Angioplasty of Advanced Atherosclerotic Severe Stenosis', 'Presentation Start Time': '', 'Presentation Time': '14:00-15:00', 'Abstract Code': 'PB0023', 'Authors': '<u>H. Mehrad</u><sup>1</sup>, A. Foletti<sup>2</sup>', 'Institution': '<sup>1</sup>Mehrad Research Lab, Division of

{'Session Time': '14:00 – 15:00 EDT', 'Session Title': 'Arterial Thromboembolism Abstracts', 'Session Date': '12.07.2020', 'Session Location': 'Virtual Meeting Room 6', 'Presentation Title': 'Effect of Photobiomodulation Therapy on Arterial Biomechanical Parameters in a Rabbit Carotid Artery Model of Early Stage Atherosclerosis', 'Presentation Start Time': '', 'Presentation Time': '14:00-15:00', 'Abstract Code': 'PB0045', 'Authors': '<u>H. Mehrad</u><sup>1</sup>, L. Kazemi<sup>2</sup>', 'Institution': '<sup>1</sup>Islamic Azad University, Tabriz Branch- Basic Sciences Faculty, Department of Physics, Tabriz, Iran, Islamic Republic of, <sup>2</sup>Islamic Azad University, Tabriz Branch- Bioengineering Faculty, Department of Medical Biomechanics, Tabriz, Iran, Islamic Republic of'}
{'Session Time': '14:00 – 15:00 EDT', 'Session Title': 'Arterial Thromboembolism Abstracts', 'Session Date': '12.07.2020', 'Session Location': 'Virtual Meeting Room 6', 'Presentation Title': 'Effect of Photodyn

...

{'Session Time': '14:00 – 15:00 EDT', 'Session Title': 'Venous Thromboembolism Abstracts', 'Session Date': '12.07.2020', 'Session Location': 'Virtual Meeting Room 6', 'Presentation Title': 'The Use of Thrombolytic Therapy in a Multidisciplinary Pulmonary Embolism Response Team', 'Presentation Start Time': '', 'Presentation Time': '14:00-15:00', 'Abstract Code': 'PB2482', 'Authors': '<u>A. Kramer</u><sup>1,2</sup>, C.S. Mortensen<sup>1,2</sup>, J.G. Schultz<sup>1,2</sup>, N. Giordano<sup>1</sup>, H. Zheng<sup>3</sup>, A. Andersen<sup>2</sup>, J.E. Nielsen-Kudsk<sup>2</sup>, C. Kabrhel<sup>1</sup>', 'Institution': '<sup>1</sup>Massachusetts General Hospital, Center for Vascular Emergencies, Department of Emergency Medicine, Boston, United States, <sup>2</sup>Aarhus University Hospital, Department of Cardiology, Aarhus N, Denmark, <sup>3</sup>Massachusetts General Hospital, Department of Medicine, Boston, United States'}
{'Session Time': '14:00 – 15:00 EDT', 'Session Title': 'Venous Throm

{'Session Time': '14:00 – 15:00 EDT', 'Session Title': "Women's Health Abstracts", 'Session Date': '12.07.2020', 'Session Location': 'Virtual Meeting Room 6', 'Presentation Title': 'Estrogens and Progestinics', 'Presentation Start Time': '', 'Presentation Time': '14:00-15:00', 'Abstract Code': '', 'Authors': '', 'Institution': ''}
{'Session Time': '14:00 – 15:00 EDT', 'Session Title': "Women's Health Abstracts", 'Session Date': '12.07.2020', 'Session Location': 'Virtual Meeting Room 6', 'Presentation Title': 'Oral Contraceptives and Venous Thromboembolism: Focus on Testing that May Enable Prediction and Assessment of the Risk', 'Presentation Start Time': '', 'Presentation Time': '14:00-15:00', 'Abstract Code': 'PB2494', 'Authors': '<u>J. Douxfils</u><sup>1,2</sup>, L. Morimont<sup>1,2</sup>, C. Bouvy<sup>2</sup>, J.-M. Dogné<sup>1</sup>', 'Institution': '<sup>1</sup>University of Namur, Department of Pharmacy, Namur, Belgium, <sup>2</sup>QUALIblood s.a., Namur, Belgium'}
{'Session Time

{'Session Time': '14:00 – 15:00 EDT', 'Session Title': "Women's Health Abstracts", 'Session Date': '12.07.2020', 'Session Location': 'Virtual Meeting Room 6', 'Presentation Title': 'Reassessment of Venous Thromboembolism Risk and Prophylaxis in Post-Delivery Period of Healthy Women', 'Presentation Start Time': '', 'Presentation Time': '14:00-15:00', 'Abstract Code': 'PB2511', 'Authors': '<u>A. Braester</u><sup>1,2</sup>, J. Borenstein<sup>2,3</sup>, A. Zverev<sup>3</sup>, E. Kukuyev<sup>4</sup>, E. Shaoul<sup>4</sup>, G. Stemer<sup>5</sup>, M. Barhoum<sup>4,6</sup>', 'Institution': "<sup>1</sup>Galilee Medical Center, Hematology, Nahariya, Israel, <sup>2</sup>Bar Ilan University, Faculty of Medicine, Safed, Israel, <sup>3</sup>Galilee Medical Center, Obstetrics, Nahariya, Israel, <sup>4</sup>Galilee Medical Center, Nahariya, Israel, <sup>5</sup>Ha'Emek Medical Center, Hematology, Afula, Israel, <sup>6</sup>Bar Ilan University, Safed, Israel"}
{'Session Time': '14:00 – 15:00 EDT', 'Sess

{'Session Time': '14:00 – 15:00 EDT', 'Session Title': "Women's Health Abstracts", 'Session Date': '12.07.2020', 'Session Location': 'Virtual Meeting Room 6', 'Presentation Title': 'Histone H4 plasma levels in women with pregnancy complications', 'Presentation Start Time': '', 'Presentation Time': '14:00-15:00', 'Abstract Code': 'PB2529', 'Authors': '<u>L. Spiezia</u><sup>1</sup>, G. Calzamatta<sup>1</sup>, E. Campello<sup>1</sup>, C. Bulato<sup>1</sup>, S. Gavasso<sup>1</sup>, M.T. Gervasi<sup>1</sup>, R. Romero<sup>2</sup>, P. Simioni<sup>1</sup>', 'Institution': '<sup>1</sup>University of Padova, Padova, Italy, <sup>2</sup>Perinatology Research Branch, Division of Obstetrics and Maternal-Fetal Medicine, Eunice Kennedy Shriver National Institute of Child Health and Human Development, National Institutes of Health, Department of Health and Human Services, Detroit, United States'}
{'Session Time': '14:00 – 15:00 EDT', 'Session Title': "Women's Health Abstracts", 'Session Date': '12.07.

{'Session Time': '14:00 – 15:00 EDT', 'Session Title': "Women's Health Abstracts", 'Session Date': '12.07.2020', 'Session Location': 'Virtual Meeting Room 6', 'Presentation Title': 'Pregnancy associated bleeding in women with inherited bleeding disorders', 'Presentation Start Time': '', 'Presentation Time': '14:00-15:00', 'Abstract Code': 'PB2554', 'Authors': '<u>O. Kaabia</u><sup>1</sup>, M. Derouiche<sup>1</sup>, R. Bouchahda<sup>1</sup>, A. Brahim<sup>2</sup>, M. Bibi<sup>1</sup>', 'Institution': '<sup>1</sup>Université de Sousse, Faculté de Médecine de Sousse, Hopital Farhat Hached, Sousse, Tunisia, <sup>2</sup>Anesthesiology, Sousse, Tunisia'}
{'Session Time': '14:00 – 15:00 EDT', 'Session Title': "Women's Health Abstracts", 'Session Date': '12.07.2020', 'Session Location': 'Virtual Meeting Room 6', 'Presentation Title': 'Women’s experience of bleeding and adverse events associated with postpartum pharmacological thromboprophylaxis: A prospective cohort study', 'Presentation Start

In [26]:
dfPresDetails

,Abstract Code,Authors,Institution,Presentation Start Time,Presentation Time,Presentation Title,Session Date,Session Location,Session Time,Session Title
0,,"[Speaker: Luigi Naldini, Italy]",,08:00,08:00-08:30,Advances in Gene Therapy and Ethical Aspects,12.07.2020,Virtual Meeting Room 1,08:00 – 08:30 EDT,Plenary Session
1,,"[Speaker: Amit Nathwani, United Kingdom]",,08:45,08:45-09:10,Progress and Obstacles to Hemophilia Gene Therapy,12.07.2020,Virtual Meeting Room 1,08:45 – 10:00 EDT,State-of-the-Art on Hemophilia and Rare Bleedi...
2,,"[Speaker: Federico Mingozzi, United States]",,09:10,09:10-09:35,Immunogenicity of Adeno-Associated Vectors,12.07.2020,Virtual Meeting Room 1,08:45 – 10:00 EDT,State-of-the-Art on Hemophilia and Rare Bleedi...
3,,"[Speaker: Sebastien Lacroix-Desmazes, France]",,09:35,09:35-10:00,Immune Responses to Factor VIII in Hemophilia,12.07.2020,Virtual Meeting Room 1,08:45 – 10:00 EDT,State-of-the-Art on Hemophilia and Rare Bleedi...
4,,"[Speaker: Giuseppe Remuzzi, Italy]",,08:45,08:45-09:10,Differential Diagnosis of Thrombotic Microangi...,12.07.2020,Virtual Meeting Room 2,08:45 – 10:00 EDT,State-of-the-Art on Thrombotic Microangiopathies
...,...,...,...,...,...,...,...,...,...,...
2519,PB2570,"<u>C. Simard</u><sup>1</sup>, M.-L. Tardif<sup...","<sup>1</sup>McGill University, General Interna...",,14:00-15:00,Hemophagocytic Lymphohistiocytosis in Pregnanc...,12.07.2020,Virtual Meeting Room 6,14:00 – 15:00 EDT,Women's Health Abstracts
2520,PB2572,"<u>B. Madan</u><sup>1</sup>, G. Gray<sup>2</su...","<sup>1</sup>StThomas' Hospital, Centre for Hae...",,14:00-15:00,Management of Pregnancy in Severe Factor V Def...,12.07.2020,Virtual Meeting Room 6,14:00 – 15:00 EDT,Women's Health Abstracts
2521,PB2573,"<u>H. Ahmadzia</u><sup>1</sup>, N. Luban<sup>2...",<sup>1</sup>George Washington School of Medici...,,14:00-15:00,Optimizing Tranexamic Acid Dosing for Hemorrha...,12.07.2020,Virtual Meeting Room 6,14:00 – 15:00 EDT,Women's Health Abstracts
2522,PB2574,"<u>M. Sassi</u>, L. Khefacha","Maternal and Neonatal Center, Monastir, Tunisia",,14:00-15:00,Obstetric Antiphospholipid Syndrome: The Chall...,12.07.2020,Virtual Meeting Room 6,14:00 – 15:00 EDT,Women's Health Abstracts


In [27]:
dfPresDetails['Authors']  = dfPresDetails['Authors'].astype('str').str.replace('[','').str.replace(']','').str.replace('<u>','').str.replace('</u>','').str.replace('<sup>','(').str.replace('</sup>',')').str.strip("'")

In [30]:
dfPresDetails['Institution'] = dfPresDetails['Institution'].astype('str').str.replace('<u>','').str.replace('</u>','').str.replace('<sup>','(').str.replace('</sup>',')').str.strip("'")

In [31]:
dfPresDetails

,Abstract Code,Authors,Institution,Presentation Start Time,Presentation Time,Presentation Title,Session Date,Session Location,Session Time,Session Title
0,,"Speaker: Luigi Naldini, Italy",,08:00,08:00-08:30,Advances in Gene Therapy and Ethical Aspects,12.07.2020,Virtual Meeting Room 1,08:00 – 08:30 EDT,Plenary Session
1,,"Speaker: Amit Nathwani, United Kingdom",,08:45,08:45-09:10,Progress and Obstacles to Hemophilia Gene Therapy,12.07.2020,Virtual Meeting Room 1,08:45 – 10:00 EDT,State-of-the-Art on Hemophilia and Rare Bleedi...
2,,"Speaker: Federico Mingozzi, United States",,09:10,09:10-09:35,Immunogenicity of Adeno-Associated Vectors,12.07.2020,Virtual Meeting Room 1,08:45 – 10:00 EDT,State-of-the-Art on Hemophilia and Rare Bleedi...
3,,"Speaker: Sebastien Lacroix-Desmazes, France",,09:35,09:35-10:00,Immune Responses to Factor VIII in Hemophilia,12.07.2020,Virtual Meeting Room 1,08:45 – 10:00 EDT,State-of-the-Art on Hemophilia and Rare Bleedi...
4,,"Speaker: Giuseppe Remuzzi, Italy",,08:45,08:45-09:10,Differential Diagnosis of Thrombotic Microangi...,12.07.2020,Virtual Meeting Room 2,08:45 – 10:00 EDT,State-of-the-Art on Thrombotic Microangiopathies
...,...,...,...,...,...,...,...,...,...,...
2519,PB2570,"C. Simard(1), M.-L. Tardif(2)","(1)McGill University, General Internal Medicin...",,14:00-15:00,Hemophagocytic Lymphohistiocytosis in Pregnanc...,12.07.2020,Virtual Meeting Room 6,14:00 – 15:00 EDT,Women's Health Abstracts
2520,PB2572,"B. Madan(1), G. Gray(2), D. Holloway(3)","(1)StThomas' Hospital, Centre for Haemostasis ...",,14:00-15:00,Management of Pregnancy in Severe Factor V Def...,12.07.2020,Virtual Meeting Room 6,14:00 – 15:00 EDT,Women's Health Abstracts
2521,PB2573,"H. Ahmadzia(1), N. Luban(2,3), S. Li(4), D. Gu...",(1)George Washington School of Medicine and He...,,14:00-15:00,Optimizing Tranexamic Acid Dosing for Hemorrha...,12.07.2020,Virtual Meeting Room 6,14:00 – 15:00 EDT,Women's Health Abstracts
2522,PB2574,"M. Sassi, L. Khefacha","Maternal and Neonatal Center, Monastir, Tunisia",,14:00-15:00,Obstetric Antiphospholipid Syndrome: The Chall...,12.07.2020,Virtual Meeting Room 6,14:00 – 15:00 EDT,Women's Health Abstracts


In [32]:
dfPresDetails.to_excel('ISTH2020.xls')